In [5]:
import numpy as np
import pandas as pd
from mlp_explainer.synthetic_data import SyntheticData
from mlp_explainer.researcher import Researcher

In [6]:
experiment_1 = Researcher("experiment_1")

In [7]:
exp1_syndata = SyntheticData("experiment_1")
exp1_syndata.create_dataset()
exp1_syndata.save()

In [8]:
experiment_1.init()

In [9]:
data = pd.read_csv('data/experiment_1_dataset.csv')

,X_0,X_1,X_2,X_3,X_4
0,-0.893371,0.371203,1.665230,-0.748551,-0.622303
1,0.828849,0.276761,1.497588,-1.132140,-0.840464
2,1.812210,0.375082,1.440779,-0.075122,2.064280
3,0.200817,0.547257,1.359619,-0.590267,-0.745104
4,-1.391886,0.387031,1.639308,-1.790589,-1.429414
...,...,...,...,...,...
123,-0.559940,-0.322942,1.240653,-0.583704,0.125002
124,0.099447,0.321348,1.440404,-0.849850,-0.774280
125,2.431583,0.062533,1.491994,-1.806327,-1.593899
126,1.369616,0.049341,1.428909,-1.051188,-0.655046
